In [ ]:
import scipy.linalg as la
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
from helper_functions.grape_functions import get_state_index
from main_grape.Grape import Grape
%pylab inline
import random as rd
import time
from IPython import display


In [ ]:
total_time = 50.0
steps = 2000

#Defining H0
qubit_state_num = 4
alpha = 0.224574
freq_ge = 3.9225#GHz
ens = np.array([ 2*np.pi*ii*(freq_ge - 0.5*(ii-1)*alpha) for ii in np.arange(qubit_state_num)])
mode_state_num = 3
qm_g1 = 2*np.pi*0.1 #GHz
mode_freq1 = 6.0 #GHz
mode_ens1 = np.array([ 2*np.pi*ii*(mode_freq1) for ii in np.arange(mode_state_num)])
qm_g2 = 2*np.pi*0.1 #GHz
mode_freq2 = 6.5 #GHz
mode_ens2 = np.array([ 2*np.pi*ii*(mode_freq2) for ii in np.arange(mode_state_num)])
state_num = qubit_state_num * (mode_state_num**2)
Q_x   = np.diag(np.sqrt(np.arange(1,qubit_state_num)),1)+np.diag(np.sqrt(np.arange(1,qubit_state_num)),-1)
Q_z   = np.diag(np.arange(0,qubit_state_num))
M_x = np.diag(np.sqrt(np.arange(1,mode_state_num)),1)+np.diag(np.sqrt(np.arange(1,mode_state_num)),-1)
I_q = np.identity(qubit_state_num)
I_m = np.identity(mode_state_num)
H_q = np.diag(ens)
H_m1 = np.diag(mode_ens1)
H_m2 = np.diag(mode_ens2)
H0 = np.kron(H_q,np.kron(I_m,I_m)) + np.kron(I_q,np.kron(H_m1,I_m)) +\
        np.kron(I_q,np.kron(I_m,H_m2)) + qm_g1*np.kron(Q_x,np.kron(M_x,I_m)) +\
        qm_g2*np.kron(Q_x,np.kron(I_m,M_x))
        
#Defining Dressed information for our multimode exp        
w_c, v_c = la.eig(H0)
dressed=[]
D= False
if D:
    for ii in range (len(v_c)):
        index=np.argmax(np.abs(v_c[:,ii]))
        if index not in dressed:
            dressed.append(index)
        else:
            temp= (np.abs(v_c[:,ii])).tolist()
            while index in dressed:

                temp.remove(max(temp))
                index2= np.argmax(np.array(temp))

                if index2<index:
                            #dressed.append(index2)
                    index=index2
                else:
                            #dressed.append(index2-1)
                    index=index2+1
            dressed.append(index)

#Defining Forbidden sates
states_h = range(3*mode_state_num**2,4*mode_state_num**2)
states_gef02 = [2,mode_state_num**2+2,2*mode_state_num**2+2]
states_gef20 = [2*mode_state_num,mode_state_num**2+2*mode_state_num,2*mode_state_num**2+2*mode_state_num]
states_forbidden_list = states_h + states_gef02 + states_gef20

g00=get_state_index(0,dressed)
g01=get_state_index(1,dressed)
g10=get_state_index(mode_state_num,dressed)
g11=get_state_index(mode_state_num+1,dressed)

#Defining Concerned states (starting states)
psi0 = [g00,g01,g10,g11]

print psi0

#Defining states to include in the drawing of occupation
states_draw_list = [g00,g01,g10,g11]
states_draw_names = ['g00','g01','g10','g11']

#Defining U (Target)



if D:
    Ct_c= np.identity(state_num) - np.outer(v_c[:,g10], np.conjugate((v_c[:,g10])))
    Ct_c= Ct_c - np.outer(v_c[:,g11], np.conjugate((v_c[:,g11])))
    
    Ct_c= Ct_c + np.outer(v_c[:,g10], np.conjugate((v_c[:,g11])))
    Ct_c= Ct_c + np.outer(v_c[:,g11], np.conjugate((v_c[:,g10])))
else:
    Ut_c = la.expm((0-1j)*total_time*H0)
    Ct_c = Ut_c.copy()

    Ct_c[g11,g10] = 1
    Ct_c[g10,g11] = 1
    Ct_c[g10,g10] = 0
    Ct_c[g11,g11] = 0
U = Ct_c

#Defining U0 (Initial)
q_identity = np.identity(qubit_state_num)
m_identity = np.identity(mode_state_num)
U0= np.kron(q_identity,np.kron(m_identity,m_identity))

#Defining control Hs
XI = np.kron(Q_x,np.kron(I_m,I_m))
ZI = np.kron(Q_z,np.kron(I_m,I_m))
Hops = [XI,ZI]
ops_max_amp = [4.0,4*np.pi]
Hnames =['x','z']

#Defining convergence parameters
max_iterations = 10000
decay = max_iterations/2
convergence = {'rate':0.002, 'update_step':10, 'max_iterations':max_iterations,\
               'conv_target':1e-3,'learning_rate_decay':decay}
penalty_coeffs = {'envelope' : 0.01, 'dc':0.01, 'dwdt':0.01,'d2wdt2':0.001*0.0001, 'forbidden':0.0}
multi_mode = {'dressed':dressed, 'vectors':v_c, 'qnum':qubit_state_num, 'mnum': mode_state_num,\
              'f':freq_ge, 'es':w_c, 'g1':qm_g1, 'D':D, 'Interpolation':False, 'Modulation':False}

In [ ]:
 #Extra options
#reg_coeffs = reg_coeffs,draw = [states_draw_list,states_draw_names] ,forbidden = states_forbidden_list,multi_mode = multi_mode,
#maxA = ops_max_amp,use_gpu = True
uks,U_f = Grape(H0,Hops,Hnames,U,total_time,steps,psi0,convergence=convergence, draw = [states_draw_list,states_draw_names],show_plots = True, 
      forbidden = states_forbidden_list, switch = False, unitary_error = 1e-6,gate = "MM_CNOT", multi_mode=multi_mode, method = 'L-BFGS-B', penalty_coeffs = penalty_coeffs, limit_dc = [1])